In [9]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable
import os 


def Normlize(Z):
    Zmax, Zmin = Z.max(axis=1), Z.min(axis=1)
    Zmean = Z.mean(axis=1)
    #按列排序
    Zmax, Zmin = Zmax.reshape(-1, 1), Zmin.reshape(-1, 1)
    Zmean = Zmean.reshape(-1, 1)
    Z = (Z - Zmean) / (Zmax - Zmin)
    return Z


def Data_Reading(Normalization=True):
    train_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainset.npy')
    train_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\train\\trainlabel.npy')
    test_x = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testset.npy')
    test_y = np.load('F:\\gitworkspace\\python\\e-nose-cnn\\test\\testlabel.npy')

    # Normalization
    train_x_Normed = Normlize(train_x)
    test_x_Normed = Normlize(test_x)

    # xlsx to tensor
    if Normalization:
        train_x = torch.from_numpy(train_x_Normed).type(torch.cuda.FloatTensor)
        test_x = torch.from_numpy(test_x_Normed).type(torch.cuda.FloatTensor)
        train_y = torch.from_numpy(train_y).type(torch.int64)
        test_y = torch.from_numpy(test_y).type(torch.int64)

    else:
        train_x = torch.from_numpy(train_x).type(torch.FloatTensor)
        test_x = torch.from_numpy(test_x).type(torch.FloatTensor)
        train_y = torch.from_numpy(train_y).type(torch.int64)
        test_y = torch.from_numpy(test_y).type(torch.int64)
    # reshape
    # train_x = train_x.view(410, 10, 1, 120)
    # test_x = test_x.view(174, 10, 1, 120)
    train_x = train_x.view(408, 10, 1, 120)
    test_x = test_x.view(202, 10, 1, 120)

    return train_x, test_x, train_y, test_y


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(10,6,(1,4),stride=(1,1))#(10 - 10)/1 + 1 = 1   (120 - 4)/1 + 1 = 117 
        self.conv2 = nn.Conv2d(6,10,(1,3),stride=(1,2))#(1 - 1)/1 + 1 = 1   (117 - 3)/2 + 1 = 58 116/2= 58
        # self.conv3 = nn.Conv2d(6,10,(1,2),stride=(1,1))
        self.fc1 = nn.Linear(10*1*58,4)

       
        # self.conv1 = nn.Conv2d(1,6,(10,4),stride=(1,2))#59
        # self.conv2 = nn.Conv2d(6,10,(1,3),stride=(1,1))#57      
        # self.fc1 = nn.Linear(10*1*57,4)

        # self.fc2 = nn.Linear(342,4)
        # self.fc3 = nn.Linear(85,4)
        # self.fc4 = nn.Linear(64,4)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.xavier_uniform_(m.weight)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.01)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        # x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        # x = F.logsigmoid(self.fc1(x))
        # x = F.logsigmoid(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = self.fc1(x)
        # return x
        return x#F.softmax(x,dim=0)


if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#cuda:0

    print(device)

    cnn = Net()
    print(cnn)
    cnn.to(device)

    #sgd -> stochastic gradient descent
    optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.8)
    loss_func = nn.CrossEntropyLoss()

    train_x, test_x, train_y, test_y = Data_Reading(Normalization=1)
    train_y = train_y.squeeze()
    test_y = test_y.squeeze()
    train_x = train_x.to(device)
    test_x = test_x.to(device)
    train_y = train_y.to(device)
    test_y = test_y.to(device)

    

    #train
    sum = 0

    batch_size = 17
    step = 10
    tr_x = Variable(train_x)
    tr_y = Variable(train_y)
    for epoch in range(100):
        running_loss = 0.0
        for i in range(0,(int)(len(train_x)/batch_size)):
            t_x = Variable(train_x[i*batch_size:i*batch_size+batch_size])
            t_y = Variable(train_y[i*batch_size:i*batch_size+batch_size])
            out = cnn(t_x)
            #forward
            loss = loss_func(out, t_y)
            #梯度初始化为零
            optimizer.zero_grad()
             
            #backward
            loss.backward()
            optimizer.step()

            out = cnn(tr_x)
            predicted_train = torch.max(out.data, 1)[1]
            total_train = tr_y.size(0)
            for j in range(te_y.size(0)):
                if predicted_test[j] == te_y[j]:
                    sum += 1
            
            print('total_train:{}, accuracy:{}, sum:{}'.format(total_train, sum / total_train, sum))
            sum = 0

            running_loss += loss.item()

            print('loss = {}'.format(loss.item()))

            # predicted1 = torch.max(out1.data, 1)[1]
   
        running_loss = running_loss / 24

        if (running_loss < 0.3) :
            optimizer = optim.SGD(cnn.parameters(), lr=0.0001, momentum=0.6)
        elif (running_loss < 0.2) :
            optimizer = optim.SGD(cnn.parameters(), lr=0.00001, momentum=0.4)

        # if (epoch + 1) % step == 0:
        print('Epoch[{}], loss: {:.8f}'.format(epoch + 1, running_loss))
    
    
    #test
    te_x = Variable(test_x)
    te_y = Variable(test_y)
    out1 = cnn(te_x)
    predicted_test = torch.max(out1.data, 1)[1]#.data.squeeze()
    total = te_y.size(0)

    for j in range(te_y.size(0)):
        if predicted_test[j] == te_y[j]:
            sum += 1

    print('predicted_test:{}, total:{}, accuracy:{}, sum:{}'.format(predicted_test, total, sum / total, sum))

5, sum:51
loss = 0.22040170431137085
total_train:408, accuracy:0.125, sum:51
loss = 0.30971208214759827
total_train:408, accuracy:0.125, sum:51
loss = 0.1409071683883667
total_train:408, accuracy:0.125, sum:51
loss = 0.09932813793420792
total_train:408, accuracy:0.125, sum:51
loss = 0.08981740474700928
total_train:408, accuracy:0.125, sum:51
loss = 0.18199221789836884
total_train:408, accuracy:0.125, sum:51
loss = 0.08542690426111221
total_train:408, accuracy:0.125, sum:51
loss = 0.47727125883102417
total_train:408, accuracy:0.125, sum:51
loss = 0.8227459192276001
total_train:408, accuracy:0.125, sum:51
loss = 0.9196613430976868
total_train:408, accuracy:0.125, sum:51
loss = 0.5471510291099548
total_train:408, accuracy:0.125, sum:51
loss = 0.6006810665130615
total_train:408, accuracy:0.125, sum:51
loss = 0.08161941915750504
total_train:408, accuracy:0.125, sum:51
loss = 0.297397643327713
total_train:408, accuracy:0.125, sum:51
loss = 0.052213557064533234
total_train:408, accuracy:0.125